In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
import io
%cd "/Users/rajeshprabhakarkaila/Downloads"

/Users/rajeshprabhakarkaila/Downloads


In [76]:
amznreviews=pd.read_csv("amazon.csv")

In [77]:
amznreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

In [78]:
amznreviews.review_content.head()

0    Looks durable Charging is fine tooNo complains...
1    I ordered this cable to connect my phone to An...
2    Not quite durable and sturdy,https://m.media-a...
3    Good product,long wire,Charges good,Nice,I bou...
4    Bought this instead of original apple, does th...
Name: review_content, dtype: object

In [79]:
# Conversion to lower case
amznreviews.review_content=amznreviews.review_content.str.lower()

In [80]:
import re
def clean_txt(text):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\s+)|([0-9])"," ",text).split())

In [81]:
amznreviews.review_content=[str(clean_txt(x)) for x in amznreviews.review_content]

In [82]:
import nltk
from nltk.corpus import stopwords

In [83]:
english_stopwords=set(stopwords.words("english"))

In [84]:
from nltk.tokenize import sent_tokenize,word_tokenize

In [106]:
reviewsentences=amznreviews.review_content.apply(sent_tokenize)

In [108]:
from textblob import TextBlob

In [110]:
def analyze_sentiment(text):
    analysis=TextBlob(text)
    if analysis.sentiment.polarity>0:
        return "Positive"
    elif analysis.sentiment.polarity==0:
        return "Neutral"
    else:
        return "Negative"

In [122]:
reviewsentences=pd.DataFrame(reviewsentences)

In [128]:
reviewsentences['sentiment']=[str(analyze_sentiment(str(x))) 
                              for x in reviewsentences.review_content]

In [130]:
reviewsentences.sentiment.value_counts()

sentiment
Positive    1458
Negative       7
Name: count, dtype: int64

In [134]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [136]:
Tfidf=TfidfVectorizer(max_features=50,stop_words="english")

In [144]:
reviewsentences['review_content'] = reviewsentences['review_content'].astype(str)

In [146]:
X_tfidf=Tfidf.fit_transform(reviewsentences.review_content)

In [150]:
pd.DataFrame(X_tfidf.toarray(),columns=Tfidf.vocabulary_).head(3)

,looks,charging,fine,really,fast,good,product,quality,speed,cable,...,range,easy,time,overall,watch,tv,sound,screen,camera,water
0,0.162710,0.0,0.0,0.0,0.0,0.000000,0.0,0.391169,0.0,0.601347,...,0.0,0.0,0.000000,0.18119,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.086274,0.0,0.590661,0.0,0.518874,...,0.0,0.0,0.065984,0.07817,0.0,0.0,0.0,0.137722,0.0,0.0
2,0.511616,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.501876,0.0,0.0


In [154]:
from imblearn.over_sampling import SMOTE

In [156]:
y=reviewsentences.sentiment

In [158]:
X_smote,y_smote=SMOTE().fit_resample(X_tfidf,y)

In [162]:
print(pd.DataFrame(y.value_counts()))
print(pd.DataFrame(y_smote.value_counts()))

           count
sentiment       
Positive    1458
Negative       7
           count
sentiment       
Positive    1458
Negative    1458


In [164]:
from sklearn.preprocessing import LabelEncoder

In [174]:
y_smote=LabelEncoder().fit_transform(y_smote)

In [176]:
pd.DataFrame(y_smote).value_counts()

0
0    1458
1    1458
Name: count, dtype: int64

In [178]:
from sklearn.linear_model import LogisticRegression

In [180]:
logisticmodel=LogisticRegression().fit(X_smote,y_smote)

In [182]:
logisticmodel.score(X_smote,y_smote)

0.9900548696844993

In [184]:
logistic_pred=logisticmodel.predict(X_smote) 

In [186]:
pd.crosstab(y_smote,logistic_pred)

col_0,0,1
row_0,,
0,1458,0
1,29,1429


In [188]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [190]:
treemodel=DecisionTreeClassifier().fit(X_smote,y_smote)

In [192]:
treemodel.score(X_smote,y_smote)

1.0

In [194]:
cross_val_score(treemodel,X_smote,y_smote)

array([0.98287671, 0.99656947, 1.        , 0.99828473, 0.99656947])